# Build a Dashboard Application with Plotly Dash

In this lab, you will be building a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in
real-time.

This dashboard application contains input components such as a dropdown list and a range slider to
interact with a pie chart and a scatter point chart. You will be guided to build this dashboard application via the following tasks:

* **TASK 1: Add a Launch Site Drop-down Input Component**  
* **TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown**  
* **TASK 3: Add a Range Slider to Select Payload**  
* **TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot**  

*Note: Please take screenshots of the Dashboard and save them. Further upload your notebook to github.*

The github url and the screenshots are later required in the presentation slides.

Your completed dashboard application should look like the following screenshot:

After visual analysis using the dashboard, you should be able to obtain some insights to answer the following five questions:
    
**1. Which site has the largest successful launches?**  
**2. Which site has the highest launch success rate?**  
**3. Which payload range(s) has the highest launch success rate?**  
**4. Which payload range(s) has the lowest launch success rate?**  
**5. Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?**

In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

C:\Users\pureg\AppData\Local\Temp/ipykernel_2576/1399112611.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\pureg\AppData\Local\Temp/ipykernel_2576/1399112611.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()


In [3]:
spacex_df.head(5)

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [4]:
spacex_df.groupby('Launch Site')['class'].sum()

Launch Site
CCAFS LC-40      7
CCAFS SLC-40     3
KSC LC-39A      10
VAFB SLC-4E      4
Name: class, dtype: int64

In [5]:
spacex_df.groupby('Launch Site')['class'].mean()

Launch Site
CCAFS LC-40     0.269231
CCAFS SLC-40    0.428571
KSC LC-39A      0.769231
VAFB SLC-4E     0.400000
Name: class, dtype: float64

In [6]:
filtered_df=spacex_df[spacex_df['Launch Site']=='CCAFS LC-40']
filtered_df_bysite = filtered_df.groupby(['Launch Site', 'class'])['Flight Number'].count().reset_index()
filtered_df_bysite

,Launch Site,class,Flight Number
0,CCAFS LC-40,0,19
1,CCAFS LC-40,1,7


### Plotly Dashboard 

In [7]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown', # 보라 수정: id -> sitedropdown
                                                      options=[
                                                          {'label': 'All Site', 'value':'ALL'},
                                                          {'label': 'CCAFS LC-40', 'value':'CCAFS LC-40'},
                                                          {'label': 'CCAFS SLC-40', 'value':'CCAFS SLC-40'},
                                                          {'label': 'KSC LC-39A', 'value':'KSC LC-39A'},
                                                          {'label': 'VAFB SLC-4E', 'value':'VAFB SLC-4E'},
                                                      ],
                                                      value='ALL',
                                                      placeholder="Select a Launch Site here",
                                                      searchable=True
                                                      ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',min=0,max=10000,step=1000,value=[min_payload,max_payload]),
                                

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
               Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    
    filtered_df = spacex_df
    
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class',
        names='Launch Site',
        title='Total Success Launches By Site')
        return fig

    else: # return the outcomes piechart for a selected site
        filtered_df=spacex_df[spacex_df['Launch Site']==entered_site]
        filtered_df_bysite = filtered_df.groupby(['Launch Site', 'class'])['Flight Number'].count().reset_index()
        fig=px.pie(filtered_df_bysite, values='Flight Number',names='class',title='Total Success Launches for site ' + entered_site)
        return fig
        

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart',component_property='figure'),
            [Input(component_id='site-dropdown',component_property='value'),
             Input(component_id='payload-slider',component_property='value')])

def scatter(site,payload):
    low, high = (payload[0],payload[1])
    mask=spacex_df[spacex_df['Payload Mass (kg)'].between(low,high)]
    if site=='ALL':
        fig=px.scatter(mask,x='Payload Mass (kg)',y='class',color='Booster Version Category',title='Correlation between Payload and Success for all Sites')
        return fig
    else:
        mask_filtered=mask[mask['Launch Site']==site]
        fig=px.scatter(mask_filtered,x='Payload Mass (kg)',y='class',color='Booster Version Category',title='Correlation between Payload and Success for Sites ' + site)
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Mar/2023 15:02:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2023 15:02:34] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2023 15:02:34] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2023 15:02:34] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2023 15:02:34] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2023 15:02:34] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2023 15:02:34] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2023 15:02:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2023 15:02:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2023 15:02:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2023 15:02:37] "POST /_dash-